notebook for code playground

In [215]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import copy

In [216]:
dataframe = pd.read_excel('./Input.xlsx')

df = copy.deepcopy(dataframe)

In [217]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


# define a function to extract the content of a link
def extract_content(url):
    # send an HTTP GET request to the website
    response = requests.get(url)

    # parse the HTML content of the website using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # if the page returns 404
    if soup.find('div', class_='td-404-title'):
        print(f'{url} 404 Error: Page not found')
        return None

    # extract the content of the article
    content = ""
    try:
        for p in soup.find("div", {"class": "td-post-content"}).find_all("p"):
            # content += p.text.strip() + "\n"
            content += p.text.strip() + " "
    except Exception as e:
        print(f'content : {e}: {url}')
    return content





In [218]:
# add a new column to the dataframe to store the content
df['content'] = ''

# iterate over the rows of the dataframe and extract the content of each link
for i, row in df.iterrows():
    try:
        link = row['URL']
        content = extract_content(link)
        df.at[i, 'content'] = content
    except Exception as e:
         print(f'content : {e}: {link}')
# print the resulting dataframe



In [219]:
df = pd.read_excel('./content.xlsx')

In [221]:
df.isna().sum()

Unnamed: 0    0
URL_ID        0
URL           0
content       5
dtype: int64

In [222]:
df = df[df['content'] != ""]

In [223]:
df = df.dropna()

In [224]:
df.isna().sum()

Unnamed: 0    0
URL_ID        0
URL           0
content       0
dtype: int64

In [1]:
import nltk
from nltk.corpus import stopwords
stopwords_dir = 'StopWords/'
def get_stopwords():
    """
    Get all the stopwords from the StopWords folder
    """
    stopwords_files = ['StopWords_Auditor.txt',
                       'StopWords_Currencies.txt',
                       'StopWords_DatesandNumbers.txt',
                       'StopWords_Generic.txt',
                       'StopWords_GenericLong.txt',
                       'StopWords_Geographic.txt',
                       'StopWords_Names.txt']
                       
    stopwords = set()
    for filename in stopwords_files:
        with open(stopwords_dir + filename, 'r', encoding='latin-1', errors='replace') as file:
            stopwords.update([word.strip() for word in file])
    # print(stopwords)
    return stopwords



In [226]:
def get_positive_words(stop_words):
    """
    Get all the positive words from the MasterDictionary folder
    """
    with open('MasterDictionary/positive-words.txt', 'r') as file:
        positive_words = [word.strip() for word in file.readlines()]

    # Remove stop words from the dictionaries
    pos_dict = {word for word in positive_words if word not in stop_words}
    return set(pos_dict)

def get_negative_words(stop_words):
    """
    Get all the negative words from the MasterDictionary folder
    """
    with open('MasterDictionary/negative-words.txt', 'r') as file:
        negative_words = [word.strip() for word in file.readlines()]

    neg_dict = {word for word in negative_words if word not in stop_words}
    return set(neg_dict)



In [227]:
def calculate_sentiment_scores(text, pos_dict, neg_dict):
    # Tokenize the text
    tokens = nltk.word_tokenize(text.lower())
    
    # Calculate positive and negative scores
    pos_score = sum(1 for word in tokens if word in pos_dict)
    neg_score = sum(-1 for word in tokens if word in neg_dict) * -1
    
    # Calculate polarity score
    polarity_score = (pos_score - neg_score) / (pos_score + neg_score + 0.000001)
    
    # Calculate subjectivity score
    subjectivity_score = (pos_score + abs(neg_score)) / (len(tokens) + 0.000001)
    
    return pos_score, neg_score, polarity_score, subjectivity_score

In [228]:
import string
def clean_text(text):
    """
    This function cleans the given text by removing punctuations and stop words.
    """
    stop_words = set(stopwords.words('english'))
    cleaned_text = ""
    for word in nltk.word_tokenize(text):
        word = word.lower()
        if word not in stop_words and word not in string.punctuation:
            cleaned_text += word + " "
    return cleaned_text.strip()

In [229]:
def syllable_count(words):
    vowels = ['a', 'e', 'i', 'o', 'u']
    count = 0
    for word in words:
        if word[-2:] == 'es' or word[-2:] == 'ed':
            continue
        word_count = 0
        for char in word:
            if char.lower() in vowels:
                word_count += 1
            count += word_count
    return count

In [230]:
def average_word_length(words):
    total_char = 0
    for word in words:
        total_char += len(word)
    return total_char/len(words)

In [231]:
import re

def personal_pronouns(text):
    pattern = r'\b(I|we|my|ours|us)\b(?!\b[A-Z]{2}\b)'
    matches = re.findall(pattern, text)
    return len(matches)

In [232]:
df['POSITIVE SCORE'] = 0
df['NEGATIVE SCORE'] = 0
df['POLARITY SCORE'] = 0
df['SUBJECTIVITY SCORE'] = 0
df['AVG SENTENCE LENGTH'] = 0
df['PERCENTAGE OF COMPLEX WORDS'] = 0
df['FOG INDEX'] = 0
df['AVG NUMBER OF WORDS PER SENTENCE'] = 0
df['COMPLEX WORD COUNT'] = 0
df['WORD COUNT'] = 0
df['SYLLABLE PER WORD'] = 0
df['PERSONAL PRONOUNS'] = 0
df['AVG WORD LENGTH'] = 0


In [235]:
stop_words = get_stopwords()
pos_dict = get_positive_words(stop_words=stop_words)
neg_dict = get_negative_words(stop_words=stop_words)

In [236]:
for i, row in df.iterrows():
    text = row['content']
    sentences_uncleaned = nltk.sent_tokenize(text)
    words_uncleaned = nltk.word_tokenize(text)
    cleaned_text = clean_text(text)
    sentences_cleaned = nltk.sent_tokenize(cleaned_text)
    words_cleaned = nltk.word_tokenize(cleaned_text)
    pos,neg, pol, sub = calculate_sentiment_scores(text,pos_dict,neg_dict)
    avg_sent_len = len(words_cleaned)/len(sentences_uncleaned)
    comp_word_count = sum([1 for word in words_cleaned if len(word) > 2])
    per_comp_words = comp_word_count/len(words_cleaned)
    fog = 0.4 * (avg_sent_len + per_comp_words)
    avg_words_per_sent = len(words_cleaned)/len(sentences_uncleaned)
    syl_per_word = syllable_count(words_cleaned)
    per_pro = personal_pronouns(text)
    avg_word_len = average_word_length(words_cleaned)

    df.at[i,'POSITIVE SCORE'] = pos
    df.at[i,'NEGATIVE SCORE'] = neg
    df.at[i,'POLARITY SCORE'] = pol
    df.at[i,'SUBJECTIVITY SCORE'] = sub
    df.at[i,'AVG SENTENCE LENGTH'] = avg_sent_len
    df.at[i,'PERCENTAGE OF COMPLEX WORDS'] = per_comp_words
    df.at[i,'FOG INDEX'] = fog
    df.at[i,'AVG NUMBER OF WORDS PER SENTENCE'] = avg_words_per_sent
    df.at[i,'WORD COUNT'] = len(words_cleaned)
    df.at[i,'SYLLABLE PER WORD'] = syl_per_word
    df.at[i,'PERSONAL PRONOUNS'] = per_pro
    df.at[i,'AVG WORD LENGTH'] = avg_word_len
    df.at[i,'COMPLEX WORD COUNT'] = comp_word_count


    

In [238]:
df.drop(columns=['Unnamed: 0','content'],inplace=True)

In [241]:
df.reset_index(drop=True)

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,66,34,0.320000,0.050226,15.333333,0.951304,6.513855,15.333333,1094,1150,12669,1,7.137391
1,38,https://insights.blackcoffer.com/what-if-the-c...,60,38,0.224490,0.060197,9.734177,0.932380,4.266623,9.734177,717,769,6658,6,6.356307
2,39,https://insights.blackcoffer.com/what-jobs-wil...,65,38,0.262136,0.054239,11.857143,0.968876,5.130407,11.857143,965,996,10726,2,7.183735
3,41,https://insights.blackcoffer.com/will-ai-repla...,58,23,0.432099,0.042100,12.973684,0.943205,5.566756,12.973684,930,986,9473,12,6.584178
4,42,https://insights.blackcoffer.com/man-and-machi...,44,23,0.313433,0.047518,12.327273,0.942478,5.307900,12.327273,639,678,6435,18,6.560472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,145,https://insights.blackcoffer.com/blockchain-in...,11,11,0.000000,0.028025,11.878788,0.987245,5.146413,11.878788,387,392,3903,2,7.104592
105,146,https://insights.blackcoffer.com/blockchain-fo...,22,27,-0.102041,0.050000,11.062500,0.979284,4.816714,11.062500,520,531,5426,9,7.143126
106,147,https://insights.blackcoffer.com/the-future-of...,36,12,0.500000,0.039344,13.285714,0.961598,5.698925,13.285714,626,651,6460,1,6.826421
107,148,https://insights.blackcoffer.com/big-data-anal...,26,46,-0.277778,0.057371,10.338462,0.973214,4.524670,10.338462,654,672,6779,2,6.678571


In [242]:
df.to_excel('./output.xlsx')